# 处理chinamutualfundnav.csv内的数据

`data/raw/chinamutualfundnav.csv` 文件太大的加载不出来，按照每个股票的code分为多个不同的文件处理

- 按照code分为多个文件
- 即哪个日期转换为正常日期格式

转化完的股票数据存放在`data/processed/` 下

In [1]:
import pandas as pd
import os
from tqdm import tqdm

# 存放处理完的数据
dest_dist = 'data/processed_fund/'

# 读取数据
data = pd.read_csv('data/raw/chinamutualfundnav.csv')

# 创建文件夹如果不存在
if not os.path.exists(dest_dist):
    os.makedirs(dest_dist)

# 记录元数据，文件名
meta_data = pd.DataFrame(columns=['code', 'start_date', 'end_date'])

# 按照f_info_code分组，存放在不同文件内
for code, group in data.groupby('f_info_windcode'):
    # 把price_date转化为日期格式
    group['price_date'] = pd.to_datetime(group['price_date'], format='%Y%m%d')
    # 确保数据按照日期排序
    group = group.sort_values('price_date')
    # 计算收益率
    group['return'] = group['f_nav_adjusted'].pct_change()
    # 保存数据
    group.to_csv(f'{dest_dist}{code}.csv', index=False)
    # 保存元数据
    meta_data = pd.concat([meta_data, pd.DataFrame(
        {'code': [code], 'start_date': [group['price_date'].iloc[0]], 'end_date': [group['price_date'].iloc[-1]]})],
                          ignore_index=True)

# 保存元数据
meta_data.to_csv(f'{dest_dist}meta_data.csv', index=False)

/var/folders/gy/v9qc5w_13rd41qstcmbm_l840000gn/T/ipykernel_51702/399483020.py:8: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/raw/chinamutualfundnav.csv')
/var/folders/gy/v9qc5w_13rd41qstcmbm_l840000gn/T/ipykernel_51702/399483020.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  meta_data = pd.concat([meta_data, pd.DataFrame(
/var/folders/gy/v9qc5w_13rd41qstcmbm_l840000gn/T/ipykernel_51702/399483020.py:24: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  group['ret

# 整理reformat中债分段指数数据

In [13]:
import pandas as pd

bond_return_index = pd.read_excel("data/raw/债券指数收益率.xlsx", header=1)  # 读取中债指数收益率数据
bond_return_index.rename(columns={bond_return_index.columns[0]: "日期"}, inplace=True)  # 重命名日期列

bond_return_index.head()

,日期,中债-国开行债券总财富(总值)指数,中债-国开行债券总净价(总值)指数,中债-国开行债券总全价(总值)指数,中债-国开行债券总财富(1年以下)指数,中债-国开行债券总净价(1年以下)指数,中债-国开行债券总全价(1年以下)指数,中债-国开行债券总财富(1-3年)指数,中债-国开行债券总净价(1-3年)指数,中债-国开行债券总全价(1-3年)指数,...,中债-企业债AAA净价(10年以上)指数,中债-企业债AAA全价(10年以上)指数,中证转债,中证国债(净),中证国债,中证国债(利),货币市场基金指数,中债-高收益企业债财富(总值)指数,中证800,沪深300
0,2010-01-04,0.000075,-0.037715,0.000094,0.013808,-0.016173,0.013788,0.009045,-0.023705,0.008976,...,-0.023245,0.031194,-0.353514,-0.020050,0.023546,0.039066,0.010468,0.042376,-0.738044,-1.131392
1,2010-01-05,0.036743,0.027805,0.036751,0.005310,-0.002238,0.005309,0.009444,0.001419,0.009554,...,0.037965,0.050577,-0.306721,-0.020054,-0.007847,0.007810,0.003469,0.052351,0.868046,0.814912
2,2010-01-06,0.015204,0.005992,-0.021046,0.009558,0.002136,0.009504,0.049776,0.042152,0.049792,...,0.026498,0.039318,0.478320,-0.020058,-0.015695,0.007809,0.002940,0.031339,-0.515759,-0.626003
3,2010-01-07,0.006547,-0.002947,0.006484,0.002941,-0.004476,0.002997,0.008958,0.000810,0.008970,...,-0.037155,-0.022899,0.167731,-0.020062,NaN,0.015618,0.003364,0.035818,-1.988454,-1.984089
4,2010-01-08,-0.029648,-0.039387,-0.048583,-0.010047,-0.017703,-0.010017,-0.015676,-0.024004,-0.032403,...,-0.145417,-0.128616,-0.073485,NaN,0.007849,0.007808,0.003885,-0.035714,0.483902,0.249866


In [14]:
# 新建一个dataframe，只保存我们想要的数据
new_df = pd.DataFrame()

# 将 “中债-国开行债券总财富指数” 当作 “中债分段指数”
new_df[["日期", "1年以下", "1-3年", "3-5年", "5-7年", "7-10年", "10年以上"]] \
    = bond_return_index[["日期", "中债-国开行债券总财富(1年以下)指数", "中债-国开行债券总财富(1-3年)指数",
                         "中债-国开行债券总财富(3-5年)指数", "中债-国开行债券总财富(5-7年)指数",
                         "中债-国开行债券总财富(7-10年)指数", "中债-国开行债券总财富(10年以上)指数"]]

# 保存数据，如果文件夹不存在则创建文件夹
import os

if not os.path.exists("data/processed_indexes/"):
    os.makedirs("data/processed_indexes/")

new_df.to_csv("data/processed_indexes/中债分段指数.csv", index=False)  # 保存数据

new_df.head()

,日期,1年以下,1-3年,3-5年,5-7年,7-10年,10年以上
0,2010-01-04,0.013808,0.009045,0.055759,0.014249,-0.125804,0.020254
1,2010-01-05,0.005310,0.009444,-0.005741,0.048008,0.110844,0.088710
2,2010-01-06,0.009558,0.049776,0.028861,-0.022398,-0.028409,0.035136
3,2010-01-07,0.002941,0.008958,0.000306,-0.004673,0.028642,-0.003743
4,2010-01-08,-0.010047,-0.015676,-0.029235,0.020401,-0.059286,-0.102135


# 把需要的数据放到 `data/combined/` 目录下

需要的数据有：
1. 基金的日收益率数据
2. 中期国债分段（<1y总财富,1-3y总财富,3-5y总财富,5-7y总财富,7-10y总财富,>10y总财富）收益率数据

先检查 `data/raw/基金筛选.xlsx` 文件中的基金是否都有其数据

In [ ]:
import pandas as pd

fund_selection = pd.read_excel('data/raw/基金筛选.xlsx')
not_found = []
for code in fund_selection['代码']:
    try:
        daily_return = pd.read_csv(f'data/processed_fund/{code}.csv')
    except FileNotFoundError:
        not_found.append(code)
        print(f'{code} not found')

not_found.__len__()

以下基金在基金筛选中存在，但是在股票收益率数据中没有，一共138只基金数据，占总比例的6.94%，原因未知。我们暂时舍弃这些数据：

`['021743.OF', '012489.OF', '020490.OF', '012115.OF', '013875.OF', '015433.OF', '018657.OF', '020123.OF', '020504.OF', '020556.OF', '020833.OF', '020834.OF', '020927.OF', '021127.OF', '021128.OF', '021266.OF', '021435.OF', '021436.OF', '021628.OF', '021677.OF', '021835.OF', '021836.OF', '021921.OF', '021928.OF', '021929.OF', '021941.OF', '021942.OF', '022040.OF', '022064.OF', '022065.OF', '022066.OF', '022086.OF', '022103.OF', '022109.OF', '022113.OF', '022118.OF', '022128.OF', '022133.OF', '022134.OF', '022135.OF', '022136.OF', '022142.OF', '022143.OF', '022155.OF', '022162.OF', '022181.OF', '022187.OF', '022207.OF', '022217.OF', '022218.OF', '022220.OF', '022227.OF', '022228.OF', '022231.OF', '022234.OF', '022241.OF', '022242.OF', '022244.OF', '022245.OF', '022246.OF', '022247.OF', '022256.OF', '022257.OF', '022265.OF', '022266.OF', '022271.OF', '022279.OF', '022289.OF', '022295.OF', '022315.OF', '022316.OF', '022317.OF', '022329.OF', '022360.OF', '022378.OF', '022379.OF', '022380.OF', '022401.OF', '022403.OF', '022404.OF', '022407.OF', '022419.OF', '022420.OF', '022477.OF', '022479.OF', '022480.OF', '022515.OF', '022536.OF', '022542.OF', '022564.OF', '022574.OF', '022584.OF', '022585.OF', '022602.OF', '022617.OF', '022618.OF', '022639.OF', '022640.OF', '022656.OF', '022659.OF', '022662.OF', '022667.OF', '022721.OF', '022722.OF', '022736.OF', '022737.OF', '022738.OF', '022761.OF', '022766.OF', '022767.OF', '022775.OF', '022786.OF', '022787.OF', '022798.OF', '022799.OF', '022806.OF', '022809.OF', '022828.OF', '022870.OF', '022871.OF', '022872.OF', '022874.OF', '022875.OF', '022876.OF', '022877.OF', '022878.OF', '022968.OF', '022993.OF', '022994.OF', '023012.OF', '023015.OF', '023018.OF', '023019.OF', '023072.OF', '023085.OF', '023091.OF', '023092.OF', 'H202108.OF']`

### 合并表格
为了缩减数据，我们只截取2010-01-04到2024-12-31之间的数据，因为我们只有这段时间的“中债分段指数”

同时将中债分段指数合并到基金数据中

In [1]:
import warnings
from tqdm import tqdm
import pandas as pd
import os

# 忽略警告
warnings.simplefilter(action='ignore', category=FutureWarning)

bond_index = pd.read_csv('data/processed_indexes/中债分段指数.csv')
fund_selection = pd.read_excel('data/raw/基金筛选.xlsx')

cache = {}
print("reading data")
for code in tqdm(fund_selection['代码']):
    try:
        cache[code] = pd.read_csv(f'data/processed_fund/{code}.csv')
    except FileNotFoundError:
        continue

if not os.path.exists("data/combined/"):
    os.makedirs("data/combined/")

print("processing data")
for code in tqdm(fund_selection['代码']):
    df = pd.DataFrame(columns=['日期', '回报', '1年以下', '1-3年', '3-5年', '5-7年', '7-10年', '10年以上'])
    fund = cache.get(code)
    if fund is None:
        continue

    for date in pd.date_range('2010-01-04', '2024-12-31'):
        index = bond_index[bond_index['日期'] == date.strftime('%Y-%m-%d')]
        fund_on_date = fund[fund['price_date'] == date.strftime('%Y-%m-%d')]

        if not index.empty and not fund_on_date.empty:
            new_row = pd.DataFrame({
                '日期': [date],
                '回报': [fund_on_date['return'].values[0]],
                '1年以下': [index['1年以下'].values[0]],
                '1-3年': [index['1-3年'].values[0]],
                '3-5年': [index['3-5年'].values[0]],
                '5-7年': [index['5-7年'].values[0]],
                '7-10年': [index['7-10年'].values[0]],
                '10年以上': [index['10年以上'].values[0]]
            })
            df = pd.concat([df, new_row], ignore_index=True)
    
    df['回报'] = df['回报'] * 100 # 转化为百分比
    df.to_csv(f'data/combined/{code}.csv', index=False)

reading data


100%|██████████| 1988/1988 [00:01<00:00, 1059.41it/s]


processing data


100%|██████████| 1988/1988 [43:19<00:00,  1.31s/it]
